### Streaming-Data-Present-Example

Git Repository : https://github.com/MrMaper/Streaming-Data-Present

For More Info Please Check The Git Page :)


In [ ]:
!pip install pyspark
!pip install findspark

In [6]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'C:\\bigdata\\spark3'

In [7]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

spark = SparkSession.builder.getOrCreate()

In [8]:
df = spark.read.csv("paysim.csv", header=True, inferSchema=True)

In [9]:
df.columns

['step',
 'type',
 'amount',
 'nameOrig',
 'oldbalanceOrg',
 'newbalanceOrig',
 'nameDest',
 'oldbalanceDest',
 'newbalanceDest',
 'isFraud',
 'isFlaggedFraud']

In [10]:
df = df.drop("isFraud", "isFlaggedFraud")

In [11]:
df.show(5)

+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+
|step|    type|  amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|
+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+
|   1| PAYMENT| 9839.64|C1231006815|     170136.0|     160296.36|M1979787155|           0.0|           0.0|
|   1| PAYMENT| 1864.28|C1666544295|      21249.0|      19384.72|M2044282225|           0.0|           0.0|
|   1|TRANSFER|   181.0|C1305486145|        181.0|           0.0| C553264065|           0.0|           0.0|
|   1|CASH_OUT|   181.0| C840083671|        181.0|           0.0|  C38997010|       21182.0|           0.0|
|   1| PAYMENT|11668.14|C2048537720|      41554.0|      29885.86|M1230701703|           0.0|           0.0|
+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+
only showing top 5 rows



In [12]:
df.groupBy("step").count().show(3)

+----+-----+
|step|count|
+----+-----+
| 148|   12|
| 463|   10|
| 471| 2620|
+----+-----+
only showing top 3 rows



This Section has been commented , because it has been run once before and we do not want to run it again

In [25]:
#%%time
#steps = df.select("step").distinct().collect()
#for step in steps[:]:
#    _df = df.where(f"step = {step[0]}")
#    #by adding coalesce(1) we save the dataframe to one file
#    _df.coalesce(1).write.mode("append").option("header", "true").csv("data/paysim")

In [13]:
!cd data/paysim/ && dir

 Volume in drive C has no label.
 Volume Serial Number is 3CAA-AC15

 Directory of C:\Users\Mr_Maper\Desktop\ML Presentation\data\paysim

01/14/2022  06:26 PM    <DIR>          .
01/14/2022  06:26 PM    <DIR>          ..
01/14/2022  02:06 PM            17,764 .part-00000-0163355b-4eeb-4fea-b840-14a1fae7a650-c000.csv.crc
01/14/2022  06:25 PM                20 .part-00000-0187413a-c9de-436a-bec7-0c34319699b4-c000.csv.crc
01/14/2022  01:44 PM            15,240 .part-00000-018a23d4-d243-4bcd-8df4-db9a95047c27-c000.csv.crc
01/14/2022  02:34 PM                16 .part-00000-018ec052-ff8b-4403-b4a5-f9ab6ff57334-c000.csv.crc
01/14/2022  02:25 PM                16 .part-00000-01f24024-aaeb-4ee2-9d99-e80442976053-c000.csv.crc
01/14/2022  01:37 PM                20 .part-00000-0274581d-46c1-4086-984d-d175d819d564-c000.csv.crc
01/14/2022  06:08 PM                16 .part-00000-0292f308-f434-4a61-bec6-f482028216a6-c000.csv.crc
01/14/2022  06:02 PM                16 .part-00000-02ad0ca7-f9f7-43bc-94

In [15]:
part = spark.read.csv(
    "data/paysim/part-00000-ffd9f40c-2a92-4cd8-896c-1affa492a34e-c000.csv",
    header=True,
    inferSchema=True,
)

In [17]:
part.groupBy("step").count().show()

+----+-----+
|step|count|
+----+-----+
| 150|   14|
+----+-----+



In [19]:
dataSchema = part.schema

In [20]:
dataSchema

StructType(List(StructField(step,IntegerType,true),StructField(type,StringType,true),StructField(amount,DoubleType,true),StructField(nameOrig,StringType,true),StructField(oldbalanceOrg,DoubleType,true),StructField(newbalanceOrig,DoubleType,true),StructField(nameDest,StringType,true),StructField(oldbalanceDest,DoubleType,true),StructField(newbalanceDest,DoubleType,true)))

In [21]:
streaming = (
    spark.readStream.schema(dataSchema)
    .option("maxFilesPerTrigger", 1)
    .csv("data/paysim/")
)

In [22]:
dest_count = streaming.groupBy("nameDest").count().orderBy(F.desc("count"))

In [23]:
activityQuery = (
    dest_count.writeStream.queryName("dest_counts")
    .format("memory")
    .outputMode("complete")
    .start()
)

# include this in production
# activityQuery.awaitTermination()

import time

for x in range(50):
    _df = spark.sql(
        "SELECT * FROM dest_counts WHERE nameDest != 'nameDest' AND count >= 2"
    )
    if _df.count() > 0:
        _df.show(10)
    time.sleep(0.5)

+-----------+-----+
|   nameDest|count|
+-----------+-----+
| C319921943|    2|
| C803352127|    2|
|C1887077333|    2|
+-----------+-----+

+-----------+-----+
|   nameDest|count|
+-----------+-----+
| C319921943|    2|
| C803352127|    2|
|C1887077333|    2|
+-----------+-----+

+-----------+-----+
|   nameDest|count|
+-----------+-----+
| C319921943|    2|
| C803352127|    2|
|C1887077333|    2|
+-----------+-----+

+-----------+-----+
|   nameDest|count|
+-----------+-----+
| C319921943|    2|
| C803352127|    2|
|C1887077333|    2|
+-----------+-----+

+-----------+-----+
|   nameDest|count|
+-----------+-----+
| C319921943|    2|
| C803352127|    2|
|C1887077333|    2|
+-----------+-----+

+-----------+-----+
|   nameDest|count|
+-----------+-----+
| C319921943|    2|
| C803352127|    2|
|C1887077333|    2|
| C763794011|    2|
| C488343370|    2|
+-----------+-----+

+-----------+-----+
|   nameDest|count|
+-----------+-----+
| C319921943|    2|
| C803352127|    2|
|C1887077333| 

Check if stream is active

In [33]:
spark.streams.active[0].isActive

IndexError: list index out of range

In [34]:
activityQuery.status

{'message': 'Stopped', 'isDataAvailable': False, 'isTriggerActive': False}

If we  want to turn off the stream we'll run activityQuery.stop()

In [29]:
activityQuery.stop()